In [ ]:
import os
import datetime
#import json
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib
#from pandas_datareader import data, wb  # Testing of datareader package
#matplotlib.style.use('ggplot')

# Initial Data Setup
start_date = '2012-05-01'
start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')

In [ ]:
data_path = 'C:/Users/Bing/github/longrun/data'
vxv_file = os.path.join(data_path, 'vxv_new.csv')
vxmt_file = os.path.join(data_path, 'vxmt_new.csv')
xiv_file = os.path.join(data_path, 'xiv.csv')
vxx_file = os.path.join(data_path, 'vxx.csv')
svxy_file = os.path.join(data_path, 'svxy_new.csv')
hvi_file = os.path.join(data_path, 'svxy.csv')
spx_file = os.path.join(data_path, 'spx.csv')
dateparse = lambda x: pd.datetime.strptime(x, '%d/%m/%Y')
dateparse_dash = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')
vxv_df = pd.read_csv(vxv_file, index_col=0, parse_dates=[0], date_parser=dateparse)
vxmt_df = pd.read_csv(vxmt_file, index_col=0, parse_dates=[0], date_parser=dateparse)
vxx_df = pd.read_csv(vxx_file, index_col=0, parse_dates=[0], date_parser=dateparse)
xiv_df = pd.read_csv(xiv_file, index_col=0, parse_dates=[0], date_parser=dateparse)
svxy_df = pd.read_csv(svxy_file, index_col=0, parse_dates=[0], date_parser=dateparse_dash)
hvi_df = pd.read_csv(hvi_file, index_col=0, parse_dates=[0], date_parser=dateparse)
spx_df = pd.read_csv(spx_file, index_col=0, parse_dates=[0], date_parser=dateparse_dash)
xiv = xiv_df[['Adj Close']]
vxx = vxx_df[['Adj Close']]
svxy = svxy_df[['Adj Close']]
hvi = hvi_df[['Adj Close']]
spx = spx_df[['Adj Close']]
spx.columns = ['spx']


In [ ]:
# calc ratios and signals
data = pd.merge(vxv_df, vxmt_df, how='left', left_index=True, right_index=True)
data = data.dropna(axis=0, how='any')


# Derived vix futures ratio and moving averages below
ratio = data['VXV'].divide(data['VXMT'])
ratio.sort_index(axis='index', inplace=True)
ratio.name = 'ratio'
ma_60 = pd.Series.rolling(ratio, window=60).mean()
ma_150 = pd.Series.rolling(ratio, window=150).mean()
ma_60.name = 'ma_60'
ma_150.name = 'ma_150'

strat_df = pd.concat([ratio, ma_60, ma_150],  axis=1)
idx = strat_df.isnull().any(axis=1)
strat_df = strat_df[~idx]
strat_df = strat_df[strat_df.index >= start_date]

etf = pd.merge(svxy, xiv, how='left', left_index=True, right_index=True)
etf = pd.merge(etf, vxx, how='left', left_index=True, right_index=True)
etf = pd.merge(etf, hvi, how='left', left_index=True, right_index=True)
etf.columns = ['svxy', 'xiv', 'vxx', 'hvi']


In [ ]:
svxy_df.tail()

In [ ]:
# First strat df with all etf data and ratio
#longrun = pd.merge(etf, strat_df , how='left', left_index=True, right_index=True)

# Strategy signals can be calculated independent of underlying data first, merge signals with price data later
# 100% Long signal
strat_df['signal'] = np.where((strat_df['ma_60']<1) & (strat_df['ma_150']<1) & (strat_df['ratio'] < strat_df['ma_60']) & (strat_df['ratio'] < strat_df['ma_150']), 1, 0)

# 50% long signal
strat_df['signal_50'] = np.where((strat_df['ma_60']<1) & (strat_df['ma_150']<1) & (strat_df['ratio'] > strat_df['ma_150']) & (strat_df['ratio'] < strat_df['ma_60']), 1, 0)

# Get enter/out signal
idx = np.where(strat_df.signal[1:].values != strat_df.signal[:-1].values)[0] + 1
idx_50 = np.where(strat_df.signal_50[1:].values != strat_df.signal_50[:-1].values)[0] + 1

# Get buy/sell cells for 100% Long
if strat_df.signal[0] == 0:
    buy_idx = idx[::2]
    sell_idx = idx[1::2]
else:
    buy_idx = np.insert(idx[1::2], 0, 0)
    sell_idx = idx[::2]

strat_df['bs'] = ''
strat_df.bs[buy_idx] = 'buy'
strat_df.bs[sell_idx] = 'sell'

# Get buy/sell cells for 50% Long
if strat_df.signal_50[0] == 0:
    buy_50_idx = idx_50[::2]
    sell_50_idx = idx_50[1::2]
else:
    buy_50_idx = np.insert(idx_50[1::2], 0, 0)
    sell_50_idx = idx_50[::2]

strat_df['bs_50'] = ''
strat_df.bs_50[buy_50_idx] = 'buy'
strat_df.bs_50[sell_50_idx] = 'sell'

# Output strat_df to csv
strat_df.to_csv(os.path.join(data_path, 'strat_signal.csv'))



In [ ]:
# Start to build the longrun strat portfolio with underlying stock/etf
# First merge stock with strat_df
all_data = pd.merge(etf, strat_df , how='left', left_index=True, right_index=True)
end_date = strat_df.index.max()
longrun = all_data[(all_data.index >= start_date) & (all_data.index <=end_date)]
# etf_name = 'xiv'
etf_name = 'svxy'


In [ ]:
# Predefined portfolio variables
capital = 0
n_shares = 0
cash = 100
capital_prev = capital
n_prev = n_shares
cash_prev = cash
port_value = capital + cash
port = []

In [ ]:
# Record rebalance variables info
n_stock = []
cap_list = []
cash_list = []

In [ ]:
# Now shift the underlying price (XIV) backward 1 day to avoid look ahead bias
longrun[etf_name] = longrun[etf_name].shift(-1)
longrun[etf_name].fillna(method='ffill', inplace=True)

In [ ]:
# Below are the trade/rebalance logics for the portfolio strating from trade day
for i in np.arange(0, len(longrun.index)):
    price = longrun[etf_name].iloc[i]
    signal = longrun.signal[i]
    signal_50 = longrun.signal_50[i]
    bs = longrun.bs[i]
    bs_50 = longrun.bs_50[i]
    # update today's share and capital from yesterday
    n = n_prev
    capital = capital_prev
    cash = cash_prev

    # enter 50% vix from all cash
    if signal_50 == 1 and bs_50 == 'buy' and bs == '':
        n = cash * 0.5 / price
        capital = n * price
        cash = cash * 0.5

    # Enter 50% into 100% (invest rest of cash into xiv)
    if signal == 1 and bs == 'buy' and signal_50 == 0 and bs_50 == 'sell':
        n = n + cash / price
        capital = n * price
        cash = 0

    # Enter 100% into xiv from all cash
    if signal == 1 and bs == 'buy' and bs_50 == '':
        n = cash / price
        capital = n * price
        cash = 0

    # Unload 50% xiv from 100%
    if signal == 0 and bs == 'sell' and bs_50 == 'buy':
        n = n * 0.5
        capital = n * price
        cash = capital

    # Sell all from 50% position or 100%
    if signal_50 == 0 and bs_50 == 'sell' and bs == '':
        cash = capital + cash
        n = 0
        capital = 0

    # Sell all from 100%
    if signal == 0 and bs == 'sell' and bs_50 == '':
        cash = capital
        n = 0
        capital = 0

    # Hold long position
    if (signal == 1 and bs == '') or (signal_50 == 1 and bs_50 == ''):
        capital = n * price

    n_prev = n
    capital_prev = capital
    cash_prev = cash
    port_value = cash + capital
    port.append(port_value)
    n_stock.append(n)
    cap_list.append(capital)
    cash_list.append(cash)

In [ ]:
port_df = longrun[[etf_name]]
port_df['strat'] = port
port_df['n'] = n_stock
port_df['capital'] = cap_list
port_df['cash'] = cash_list


In [ ]:
import matplotlib.pyplot as plt
import matplotlib
from pandas_datareader import data, wb  # Testing of datareader package
matplotlib.style.use('ggplot')

In [ ]:
%matplotlib inline  

In [138]:
# Performance Calculation
perf_df = pd.merge(port_df, spx , how='left', left_index=True, right_index=True)
perf_df['strat_ret'] = perf_df['strat'].pct_change(1)
perf_df['spx_ret'] = perf_df['spx'].pct_change(1)
perf_df['ex_ret'] = perf_df['strat_ret'] - perf_df['spx_ret']
perf_df = perf_df.fillna(0)
perf_df.to_csv(os.path.join(data_path, 'performance.csv'))

total_return = perf_df['strat'][-1]/perf_df['strat'][1] - 1
daily_std = np.nanstd(perf_df['strat_ret'])
annual_std = daily_std * np.sqrt(252)
sharpe_ratio = np.sqrt(252) * np.average(perf_df['strat_ret']) / daily_std
ir = perf_df['ex_ret'].mean() / perf_df['ex_ret'].std() * np.sqrt(252)
ndays = (perf_df.index[-1] - perf_df.index[0]).days
nyears = ndays/252
year_return = np.power(1 + total_return, 1/nyears) - 1
perf_dict = {'total_return': total_return, 'yearly_return': year_return, 'yearly_std': annual_std, 'sharpe_ratio': sharpe_ratio, 'IR': ir}
perf_dict


{'IR': 0.73165103911915419,
 'sharpe_ratio': 1.2066495358254268,
 'total_return': 5.5535655725221575,
 'yearly_return': 0.25456519491204799,
 'yearly_std': 0.31042301421387158}